In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/multi_turn_rewrite/chinese_tagging/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow as tf

import numpy as np
import pprint
import logging

from pathlib import Path
from sklearn.metrics import accuracy_score

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.2
GPU Enabled: True


In [4]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

In [5]:
def align_pad(li):
  max_len = max([len(sent) for sent in li])
  for sent in li:
    if len(sent) < max_len:
      sent += [0] * (max_len - len(sent))

In [6]:
def data_generator(f_paths, params):
  for f_path in f_paths:
    with open(f_path) as f:
      print('Reading', f_path)
      for line in f:
        line = line.rstrip()
        h1, h2, q, a, t1, t2 = line.split('\t')
        char2idx_fn = lambda x: [params['char2idx'].get(c, len(params['char2idx'])) for c in list(x)]
        h1, h2, q = char2idx_fn(h1), char2idx_fn(h2), char2idx_fn(q)
        align_pad([h1, h2])
        t1 = [int(t) for t in t1]
        t2 = [int(t) for t in t2]
        align_pad([t1, t2])
        nested = ({'history': [h1, h2], 'query': q}, [t1, t2])
        yield nested

In [7]:
def dataset(is_training, params):
  _shapes = ({'history':[None, None], 'query':[None]}, [None, None])
  _types = ({'history':tf.int32, 'query': tf.int32}, tf.int32)
  _pads = ({'history':0, 'query':0}, 0)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [8]:
def bigru_encode(encoder, x, mask):
  enc_out, state_fw, state_bw = encoder(x, mask=mask)
  enc_state = tf.concat((state_fw, state_bw), axis=-1)
  return enc_out, enc_state


def forward(features, labels, mode):
  history = features['history']
  query = features['query']

  is_training = (mode == tf.estimator.ModeKeys.TRAIN)
  batch_sz = tf.shape(query)[0]

  query_mask = tf.sign(query)

  num_history = tf.shape(history)[1]
  history_len = tf.shape(history)[2]
  history = tf.reshape(history, (num_history*batch_sz, history_len))
  history_mask = tf.sign(history)
  
  
  with tf.variable_scope('Embedding'):
    embedding = tf.Variable(np.load('../vocab/char.npy'),
                            dtype=tf.float32,
                            name='fasttext_vectors')
    def embed_fn(x):
      x = tf.nn.embedding_lookup(embedding, x)
      x = tf.layers.dropout(x, params['dropout_rate'], training=is_training)
      return x
    query = embed_fn(query)
    history = embed_fn(history)
  
  
  with tf.variable_scope('Encoder'):
    encoder = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
      params['hidden_units'], return_state=True, return_sequences=True, zero_output_for_mask=True))

    query_out, query_state = bigru_encode(encoder, query, query_mask)
    history_out, history_state = bigru_encode(encoder, history, history_mask)
    history_out = tf.reshape(history_out, (batch_sz, num_history*history_len, 2*params['hidden_units']))

    h = tf.layers.dense(history_out, params['hidden_units'], params['activation'], name='fc_encode_history')
    q = tf.layers.dense(query_out, params['hidden_units'], params['activation'], name='fc_encode_query')
    h_ = tf.layers.dropout(h, params['dropout_rate'], training=is_training)
    q_ = tf.layers.dropout(q, params['dropout_rate'], training=is_training)
    h_ = tf.reshape(h, (num_history, batch_sz, history_len, params['hidden_units']))

    def gen_attn(each_history):
      align = tf.matmul(each_history, q_, transpose_b=True)

      pad = tf.fill(tf.shape(align), float('-inf'))
      mask = tf.tile(tf.expand_dims(query_mask, 1), [1, history_len, 1])
      align = tf.where(tf.equal(mask, 0), pad, align)

      align = tf.nn.softmax(align)
      x = tf.matmul(align, q_)
      return x
  
    h_ = tf.map_fn(gen_attn, h_)
    h_ = tf.reshape(h_, (num_history*batch_sz, history_len, params['hidden_units']))
    h = tf.reshape(h, (num_history*batch_sz, history_len, params['hidden_units']))

    features = tf.concat((h, h_, h*h_, h-h_), -1)
    features = tf.layers.dropout(features, params['dropout_rate'], training=is_training)
    features = tf.layers.dense(features, params['hidden_units'], params['activation'], name='fc_feature')
    features = tf.layers.dropout(features, params['dropout_rate'], training=is_training)

    encoder = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
      params['hidden_units'], return_state=True, return_sequences=True, zero_output_for_mask=True))
    x, _ = bigru_encode(encoder, features, history_mask)

    x = tf.layers.dropout(x, params['dropout_rate'], training=is_training)
    x = tf.layers.dense(x, params['hidden_units'], params['activation'])
    x = tf.squeeze(tf.layers.dense(x, 1), -1)
    x = tf.reshape(x, (batch_sz, num_history*history_len))
    return x

In [9]:
def clr(step,
        initial_learning_rate,
        maximal_learning_rate,
        step_size,
        scale_fn,
        scale_mode,):
  step = tf.cast(step, tf.float32)
  
  initial_learning_rate = tf.convert_to_tensor(
    initial_learning_rate, name='initial_learning_rate')
  dtype = initial_learning_rate.dtype
  maximal_learning_rate = tf.cast(maximal_learning_rate, dtype)
  step_size = tf.cast(step_size, dtype)
  cycle = tf.floor(1 + step / (2 * step_size))
  x = tf.abs(step / step_size - 2 * cycle + 1)

  mode_step = cycle if scale_mode == 'cycle' else step

  return initial_learning_rate + (
    maximal_learning_rate - initial_learning_rate) * tf.maximum(
      tf.cast(0, dtype), (1 - x)) * scale_fn(mode_step)


def clip_grads(loss):
  variables = tf.trainable_variables()
  pprint.pprint(variables)
  grads = tf.gradients(loss, variables)
  clipped_grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
  return zip(clipped_grads, variables)

In [10]:
def model_fn(features, labels, mode, params):
  logits = forward(features, labels, mode)
  
  history = features['history']
  shape = tf.shape(history)
  history = tf.reshape(history, (shape[0], shape[1]*shape[2]))
  history_mask = tf.cast(tf.sign(history), tf.float32)
  
  if mode == tf.estimator.ModeKeys.PREDICT:
    scores = tf.sigmoid(logits)
    scores *= history_mask
    scores = tf.cast(tf.math.greater_equal(logits, .5), tf.int32)
    return tf.estimator.EstimatorSpec(mode, predictions=scores)

  shape = tf.shape(labels)
  labels = tf.reshape(labels, (shape[0], shape[1]*shape[2]))
  labels = tf.cast(labels, tf.float32)
  loss = tf.nn.weighted_cross_entropy_with_logits(labels=labels, logits=logits, pos_weight=1.5)
  loss_op = tf.reduce_sum(loss * history_mask) / tf.reduce_sum(history_mask)
    
  if mode == tf.estimator.ModeKeys.TRAIN:
    global_step=tf.train.get_or_create_global_step()
    
    decay_lr = clr(
      step = global_step,
      initial_learning_rate = 1e-4,
      maximal_learning_rate = 8e-4,
      step_size = 8 * params['buffer_size'] // params['batch_size'],
      scale_fn=lambda x: 1 / (2.0 ** (x - 1)),
      scale_mode = 'cycle',)
    
    train_op = tf.train.AdamOptimizer(decay_lr).apply_gradients(
        clip_grads(loss_op), global_step=global_step)
    
    hook = tf.train.LoggingTensorHook({'lr': decay_lr}, every_n_iter=100)
    
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss_op, train_op=train_op, training_hooks=[hook],)
    
  if mode == tf.estimator.ModeKeys.EVAL:
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss_op)

In [11]:
params = {
  'model_dir': '../model/tagging_only_pos',
  'log_path': '../log/tagging_only_pos.txt',
  'train_path': ['../data/train_pos_tag.txt'],
  'test_path': ['../data/test_pos_tag.txt'],
  'vocab_path': '../vocab/char.txt',
  'max_len': 30,
  'activation': tf.nn.elu,
  'dropout_rate': .2,
  'hidden_units': 300,
  'clip_norm': .1,
  'buffer_size': 18986,
  'batch_size': 32,
  'num_patience': 5,
  'label_smoothing': .1,
}

In [12]:
params['char2idx'] = get_vocab(params['vocab_path'])
params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

def minimal_test(estimator):
  test_str = '成都房价是多少|不买就后悔了成都房价还有上涨空间|买不起'
  h1, h2, q = test_str.split('|')
  char2idx_fn = lambda x: [params['char2idx'].get(c, len(params['char2idx'])) for c in list(x)]
  h1_, h2_, q = char2idx_fn(h1), char2idx_fn(h2), char2idx_fn(q)
  q = q + [2]
  align_pad([h1_, h2_])
  predicted = list(estimator.predict(tf.estimator.inputs.numpy_input_fn(
    x = {'history':np.reshape([h1_, h2_], (1, 2, len(h1_))),
         'query':np.reshape(q, (1, len(q)))},
         shuffle = False)))[0]
  predicted = np.reshape(predicted, (2, -1))
  print('-'*12)
  print('minimal test')
  print('Query:', test_str)
  print('Retrieved:')
  for i, h in enumerate((h1, h2)):
    str_out = ''
    for j, c in enumerate(list(h)):
      if predicted[i, j] == 1:
        str_out += c
    print(str_out)
  print('-'*12)

In [13]:
# Create directory if not exist
Path(os.path.dirname(params['log_path'])).mkdir(exist_ok=True)
Path(params['model_dir']).mkdir(exist_ok=True, parents=True)

# Logging
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)
fh = logging.FileHandler(params['log_path'])
logger.addHandler(fh)

# Create an estimator
estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  model_dir=params['model_dir'],
  config=tf.estimator.RunConfig(
    save_checkpoints_steps=params['buffer_size']//params['batch_size']+1,
    keep_checkpoint_max=3),
  params=params)

best_recall = 0.
count = 0
tf.enable_eager_execution()

while True:
  estimator.train(input_fn=lambda: dataset(is_training=True, params=params))

  minimal_test(estimator)
  # BLEU
  labels = [label for _, label in dataset(is_training=False, params=params)]
  labels = [j.flatten() for i in labels for j in i.numpy()]

  preds = list(estimator.predict(input_fn=lambda: dataset(is_training=False, params=params)))
  assert len(labels) == len(preds)

  em = [np.array_equal(p, l) for p, l in zip(preds, labels)]
  em = np.asarray(em).mean()

  l = [num for arr in labels for num in arr]
  p = [num for arr in preds for num in arr]
  assert len(l) == len(p)
  recall = accuracy_score(l, p, sample_weight=l)
  precision = accuracy_score(l, p, sample_weight=p)

  logger.info("Recall: {:.3f} | Precision: {:.3f} | EM: {:.3f}".format(recall, precision, em))

  if recall > best_recall:
    best_recall = recall
    count = 0
  else:
    count += 1
  logger.info("Best Recall: {:.3f}".format(best_recall))

  if count == params['num_patience']:
    print(params['num_patience'], "times not improve the best result, therefore stop training")
    break

INFO:tensorflow:Using config: {'_model_dir': '../model/tagging_only_pos', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 594, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9fc090f208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized 